In [1]:
import pandas as pd

filename = "loan.csv"

loan_data = pd.read_csv(filename)

C:\Users\Bineesh\AppData\Local\Temp\ipykernel_21456\1607258213.py:5: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv(filename)


In [2]:
loan_data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

* Load data contains 111 columns

In [3]:
filename1 = "Data_Dictionary.xlsx"
data_dictionary = pd.read_excel(filename1)




In [4]:
data_dictionary.columns
loanstat = 'loanstat'
description = 'description'
data_dictionary.columns = [loanstat, description]
data_dictionary

,loanstat,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
112,verification_status,"Indicates if income was verified by LC, not ve..."
113,verified_status_joint,Indicates if the co-borrowers' joint income wa...
114,zip_code,The first 3 numbers of the zip code provided b...
115,NaN,NaN


* data dictionary contains 117 rows which corresponds to 111 columns in the load dataframe
* So need to correct this data quality issue

In [5]:
column_df = pd.DataFrame({loanstat: loan_data.columns.sort_values()})
column_df

,loanstat
0,acc_now_delinq
1,acc_open_past_24mths
2,addr_state
3,all_util
4,annual_inc
...,...
106,total_rev_hi_lim
107,url
108,verification_status
109,verification_status_joint


In [6]:
def get_column_mismatch(loan_data, data_dictionary):
    diff1 = set(loan_data.columns) - set(data_dictionary[loanstat])
    print("These are the extra columns in loan_data\n", diff1)
    diff2 = set(data_dictionary[loanstat]) - set(loan_data.columns)
    print("These are the extra info in data dictionary\n", diff2)
    return diff1, diff2
    
get_column_mismatch(loan_data, data_dictionary)

These are the extra columns in loan_data
 {'verification_status_joint', 'total_rev_hi_lim'}
These are the extra info in data dictionary
 {nan, 'verified_status_joint', 'last_fico_range_high', 'total_rev_hi_lim \xa0', 'last_fico_range_low', 'fico_range_low', 'fico_range_high'}


({'total_rev_hi_lim', 'verification_status_joint'},
 {'fico_range_high',
  'fico_range_low',
  'last_fico_range_high',
  'last_fico_range_low',
  nan,
  'total_rev_hi_lim \xa0',
  'verified_status_joint'})

* There are two columns extra in the loan data.

* There are two columns extra in the loan data.
* There is an issue with total_rev_hi_lim, ie some special characters in it
* verification_status_joint in loan_data is verified_status_joint in data dictionary
* There are nan values in the data dictionary


In [7]:
mismatched_values = {'total_rev_hi_lim \xa0': 'total_rev_hi_lim', 'verified_status_joint': 'verification_status_joint'}
data_dictionary[loanstat] = data_dictionary[loanstat].replace(mismatched_values)

get_column_mismatch(loan_data, data_dictionary)

These are the extra columns in loan_data
 set()
These are the extra info in data dictionary
 {nan, 'last_fico_range_high', 'last_fico_range_low', 'fico_range_low', 'fico_range_high'}


(set(),
 {'fico_range_high',
  'fico_range_low',
  'last_fico_range_high',
  'last_fico_range_low',
  nan})

* 'fico_range_high',
  'fico_range_low',
  'last_fico_range_high',
  'last_fico_range_low' These 4 variables to be interpreted later. These are present in data dictionary but not present in loan data. Need to check whether these variables can be derived from existing variables

In [8]:
cleaned_data_dictionary = column_df.merge(data_dictionary, on=loanstat, how='inner')
cleaned_data_dictionary.to_csv("cleaned_data_dictionary.csv", index=False)

In [9]:
cleaned_data_dictionary.set_index(loanstat)[description].to_dict()


{'acc_now_delinq': 'The number of accounts on which the borrower is now delinquent.',
 'acc_open_past_24mths': 'Number of trades opened in past 24 months.',
 'addr_state': 'The state provided by the borrower in the loan application',
 'all_util': 'Balance to credit limit on all trades',
 'annual_inc': 'The self-reported annual income provided by the borrower during registration.',
 'annual_inc_joint': 'The combined self-reported annual income provided by the co-borrowers during registration',
 'application_type': 'Indicates whether the loan is an individual application or a joint application with two co-borrowers',
 'avg_cur_bal': 'Average current balance of all accounts',
 'bc_open_to_buy': 'Total open to buy on revolving bankcards.',
 'bc_util': 'Ratio of total current balance to high credit/credit limit for all bankcard accounts.',
 'chargeoff_within_12_mths': 'Number of charge-offs within 12 months',
 'collection_recovery_fee': 'post charge off collection fee',
 'collections_12_mth

In [10]:
loan_data[data_dictionary.iloc[0,0]].value_counts()

0    39717
Name: acc_now_delinq, dtype: int64